In [68]:
import pandas as pd 
import json 
import re 
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

In [69]:
# get Bibles 
bible_dir = "/Users/amycweng/DH/SERMONS_APP/db/data/Bibles"
bibles = ['Douay-Rheims','Geneva','KJV','Tyndale','Wycliffe','Vulgate']
bible_verses = {}
for b in bibles: 
    with open(f"{bible_dir}/{b}.json",'r') as f: 
        verses = json.load(f)
    for v in verses: 
        bible_verses[f'{v} ({b})'] = verses[v]
list(bible_verses.keys())[0]


'Genesis 1.1 (Douay-Rheims)'

In [79]:
use_cols = ["token","cite_label","qp_label","sent_idx","paragraph_idx"]
df = pd.read_csv("/Users/amycweng/DH/EEPS/encodings_new/A04511_encoded.csv",
                          usecols = use_cols)
df['new_cited'] = ''
df['new_qp'] = ''
grouped_df = df.groupby('sent_idx').agg({
    'token': lambda x: ' '.join(x),  
    'cite_label':lambda x: list(set(x)),
    'new_cited': lambda x: ''.join(x),
    'qp_label':lambda x: list(set(x)),
    'new_qp': lambda x: ''.join(x)
    # 'paragraph_idx':lambda x: list(set(x))
}).reset_index()
grouped_df

,sent_idx,token,cite_label,new_cited,qp_label,new_qp
0,9,Psal. 37. Hope thou in the Lorde and kepe his ...,"[nan, ['Psal 37']]",,[nan],
1,11,Rom. v. Tribulacion bringeth pacience .,"[['Rom 5'], nan]",,[nan],
2,13,"Psal. 37. Holde the styll in the Lorde , and a...","[nan, ['Psal 37']]",,[nan],
3,24,Galat. vi . that thou mayest reape euerlastyng...,"[nan, ['Galat 6']]",,[nan],
4,27,"Ose x . So V Ve unto the righteousnesse , and ...","[['Ose 10'], nan]",,[nan],
...,...,...,...,...,...,...
121,554,Set youre affection on thynges that are aboue ...,[nan],,"[[('Colossians 3.2 (Geneva)', 0.85)]]",
122,555,And what are those thinges on earthe whiche ar...,[nan],,[nan],
123,556,"Mortify ( saith he ) your earthly mebres , for...",[nan],,"[[('Colossians 3.5 (Geneva)', 0.88)]]",
124,557,"& couetousnes , wrath , fersenes , * , cursed ...",[nan],,"[[('Mark 7.22 (Douay-Rheims)', 0.86)]]",


In [80]:
grouped_df['qp_text']  = ''
pattern = r"\('([^']+)', ([0-9.]+)\)"
for idx, qp in enumerate(grouped_df['qp_label']): 
    for qp_labels in qp: 
        if not isinstance(qp_labels,float): 
            matches = re.findall(pattern, qp_labels)
            result = [(text, float(score)) for text, score in matches]
            grouped_df['qp_text'][idx] = []
            for r in result: 
                grouped_df['qp_text'][idx].append(bible_verses[r[0]])
grouped_df

,sent_idx,token,cite_label,new_cited,qp_label,new_qp,qp_text
0,9,Psal. 37. Hope thou in the Lorde and kepe his ...,"[nan, ['Psal 37']]",,[nan],,
1,11,Rom. v. Tribulacion bringeth pacience .,"[['Rom 5'], nan]",,[nan],,
2,13,"Psal. 37. Holde the styll in the Lorde , and a...","[nan, ['Psal 37']]",,[nan],,
3,24,Galat. vi . that thou mayest reape euerlastyng...,"[nan, ['Galat 6']]",,[nan],,
4,27,"Ose x . So V Ve unto the righteousnesse , and ...","[['Ose 10'], nan]",,[nan],,
...,...,...,...,...,...,...,...
121,554,Set youre affection on thynges that are aboue ...,[nan],,"[[('Colossians 3.2 (Geneva)', 0.85)]]",,[Set your affections on things which are aboue...
122,555,And what are those thinges on earthe whiche ar...,[nan],,[nan],,
123,556,"Mortify ( saith he ) your earthly mebres , for...",[nan],,"[[('Colossians 3.5 (Geneva)', 0.88)]]",,[Mortifie therefore your members which are on ...
124,557,"& couetousnes , wrath , fersenes , * , cursed ...",[nan],,"[[('Mark 7.22 (Douay-Rheims)', 0.86)]]",,"[Thefts, covetousness, wickedness, deceit, las..."


In [81]:
new_order = ['sent_idx','token','qp_text',
             'cite_label','new_cited',
             'qp_label','new_qp']
grouped_df = grouped_df[new_order]
grouped_df

,sent_idx,token,qp_text,cite_label,new_cited,qp_label,new_qp
0,9,Psal. 37. Hope thou in the Lorde and kepe his ...,,"[nan, ['Psal 37']]",,[nan],
1,11,Rom. v. Tribulacion bringeth pacience .,,"[['Rom 5'], nan]",,[nan],
2,13,"Psal. 37. Holde the styll in the Lorde , and a...",,"[nan, ['Psal 37']]",,[nan],
3,24,Galat. vi . that thou mayest reape euerlastyng...,,"[nan, ['Galat 6']]",,[nan],
4,27,"Ose x . So V Ve unto the righteousnesse , and ...",,"[['Ose 10'], nan]",,[nan],
...,...,...,...,...,...,...,...
121,554,Set youre affection on thynges that are aboue ...,[Set your affections on things which are aboue...,[nan],,"[[('Colossians 3.2 (Geneva)', 0.85)]]",
122,555,And what are those thinges on earthe whiche ar...,,[nan],,[nan],
123,556,"Mortify ( saith he ) your earthly mebres , for...",[Mortifie therefore your members which are on ...,[nan],,"[[('Colossians 3.5 (Geneva)', 0.88)]]",
124,557,"& couetousnes , wrath , fersenes , * , cursed ...","[Thefts, covetousness, wickedness, deceit, las...",[nan],,"[[('Mark 7.22 (Douay-Rheims)', 0.86)]]",
